## Упражнение 02: Присоединиться

In [2]:
import pandas as pd
import sqlite3

## создаем подключение к базе данных с помощью библиотеки sqlite3

In [3]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

## создать новую таблицу datamart в базе данных, объединив таблицы pageviews и checker, используя только один запрос:
- таблица должна иметь следующие столбцы: uid, labname, first_commit_ts, first_view_ts
- first_commit_ts — это просто новое имя столбца timestamp из таблицы проверки, оно показывает первый коммит из конкретной лаборатории и от конкретного пользователя.
- first_view_ts — первый визит пользователя в - таблицу pageviews, временная метка, когда пользователь посетил ленту новостей
- статус = «готов» все еще должен быть фильтром
- numTrials = 1 все равно должен быть фильтром
имена лабораторий должны быть из списка: 'laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1'
- таблица должна содержать только пользователей (uid с user_*), а не администраторов
- first_commit_ts и first_view_ts следует анализировать как datetime64[ns]

In [4]:
query = """
        create table if not exists datamart as
        select checker.uid, checker.labname, checker.timestamp as first_commit_ts,
        pageviews.datetime as first_view_ts from checker
        left join pageviews on pageviews.uid = checker.uid
        where checker.uid like 'user_%' and
        status = 'ready' and numTrials = 1 and
        labname in ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
        and (pageviews.datetime = (select min(datetime) from pageviews where checker.uid = pageviews.uid) or pageviews.datetime is null)
        """

conn.execute(query)

datamart = pd.read_sql('select * from datamart', conn, parse_dates=['first_commit_ts', 'first_view_ts'])
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
...,...,...,...,...
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT
136,user_19,laba06s,2020-05-21 13:27:06.705881,2020-04-21 20:30:38.034966
137,user_23,laba06s,2020-05-21 14:29:15.709568,NaT
138,user_17,laba06,2020-05-21 15:21:31.567615,2020-04-18 10:56:55.833899


## Используя методы Pandas, создайте два фрейма данных: тестовый и контрольный.

- тест должен иметь пользователей, которые имеют значения в first_view_ts
- элемент управления должен иметь пользователей, у которых отсутствуют значения в first_view_ts
- замените отсутствующие значения в элементе управления средним значением first_view_ts тестовых пользователей, мы будем использовать это значение для будущего анализа
- сохраните обе таблицы в базе данных, они вам понадобятся в следующих упражнениях.

In [13]:
test = datamart[datamart['first_view_ts'].notnull()]
control = datamart[datamart['first_view_ts'].isnull()]
control = control.fillna(test['first_view_ts'].mean())

test.info()
control.info()

test.to_sql('test', conn)
control.to_sql('control', conn)

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 3 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 0 to 137
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


81

## закрыть соединение

In [ ]:
conn.close()